In [ ]:
import os, json
from azureml.core import Workspace, Experiment, Dataset, Datastore, ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DockerConfiguration

workspace = Workspace.from_config()

In [ ]:
#datastore = Datastore(workspace, "demostore")
datastore = workspace.get_default_datastore()
input_data = Dataset.File.from_files(datastore.path('mlops_timeseries/mlops_forecast_data2013.csv')).as_named_input('input_ds').as_mount()


In [ ]:
myenv = Environment.from_conda_specification(name="arima-env", file_path="arima-env.yml")
docker_config = DockerConfiguration(use_docker=True)

aml_compute = 'demo-cpucluster1'  # <== Compute Cluster Name
source_directory = './scripts'
script='train.py'

In [ ]:
model_name = 'arima-model-v3'
experiment_name = 'arima-mlops-remote-v3'

In [ ]:
src = ScriptRunConfig(source_directory=source_directory,
                      script=script,
                      arguments =[
                                  '--model_name', model_name,
                                  '--input', input_data, 
                                 ],
                      compute_target=aml_compute,
                      environment=myenv,
                      docker_runtime_config=docker_config)

In [ ]:
experiment = Experiment(workspace = workspace, name = experiment_name)

run = experiment.submit(config=src)
run

In [ ]:
run.wait_for_completion(show_output=True, wait_post_processing=True)

In [ ]:
os.makedirs('./models', exist_ok=True)
run.download_file('outputs/models/' + 'arima_model.pkl', output_file_path='models')
run.download_file('outputs/metric.json', output_file_path='scripts')

In [ ]:
run.get_file_names()

In [ ]:
#run.register_model('arima-model-v3', model_path='outputs/arima_model.pkl', tags=run.get_metrics(), description='arima-model-v2')